In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('train_dataset_train.csv')
data.head()

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm,time_to_under,label
0,1,40BD89EC85646EFB69E283F39C298E60,Пропуск FacePay,2402,Лефортово БКЛ,11007,Лефортово,11,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001
1,2,126727A96489CC976A8C08E5CEB00542,СК учащегося 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011
2,3,D28CE6A9E0E5B6D213470A97CFF32485,БСК дружинника г.Москвы,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022
3,4,015DA44B523C062B5BFEFF3FB0E64B9E,30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022
4,5,95B19C6F3A504727AC3EA56EB7E3E80F,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017


In [3]:
data.nunique()

id                1091021
ticket_id          335533
ticket_type_nm         60
entrance_id           426
entrance_nm           426
station_id            276
station_nm            245
line_id                16
line_nm                16
pass_dttm          297040
time_to_under       60514
label                 276
dtype: int64

In [4]:
data.isna().sum()

id                0
ticket_id         0
ticket_type_nm    0
entrance_id       0
entrance_nm       0
station_id        0
station_nm        0
line_id           0
line_nm           0
pass_dttm         0
time_to_under     0
label             0
dtype: int64

In [5]:
data.shape

(1091021, 12)

In [6]:
# data.pass_dttm = pd.to_datetime(data.pass_dttm)
#data["hour"] = data.pass_dttm.dt.hour
#data["minute"] = data.pass_dttm.dt.minute
#data["second"] = data.pass_dttm.dt.second
#data['day'] = data.pass_dttm.dt.weekday
#data["is_weekend"] = data.pass_dttm.dt.weekday.isin([5, 6]) * 1
#data

In [7]:
X = data.iloc[:, 2:]
y = data[data.columns[-1]]
y.head()

0    8001
1    9011
2    7022
3    2022
4    2017
Name: label, dtype: int64

In [8]:
X = X.drop(columns=['pass_dttm','label','entrance_nm','station_nm','line_nm','pass_dttm','time_to_under'])
X.head()

,ticket_type_nm,entrance_id,station_id,line_id
0,Пропуск FacePay,2402,11007,11
1,СК учащегося 30 дней,110,2006,2
2,БСК дружинника г.Москвы,110,2006,2
3,30 дней,110,2006,2
4,КОШЕЛЕК,110,2006,2


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, 
                                                       train_size=0.8, 
                                                       random_state=42)

In [11]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [12]:
import catboost

In [13]:
categorical_indices = np.array([0,1,2,3])

In [14]:
boosting_model = catboost.CatBoostClassifier(n_estimators=50,cat_features=categorical_indices)

boosting_model.fit(X_train, y_train)

Learning rate set to 0.5
0:	learn: 5.4495316	total: 24m 14s	remaining: 19h 48m 5s
1:	learn: 5.7268886	total: 50m 50s	remaining: 20h 20m 22s
2:	learn: 15.6000017	total: 1h 17m 15s	remaining: 20h 10m 22s
3:	learn: 45.5880425	total: 1h 43m 23s	remaining: 19h 49m
4:	learn: 65.7915163	total: 2h 9m 59s	remaining: 19h 29m 54s
5:	learn: 79.6771892	total: 2h 36m 27s	remaining: 19h 7m 23s
6:	learn: 88.5899204	total: 3h 2m 41s	remaining: 18h 42m 16s
7:	learn: 96.7829446	total: 3h 29m 7s	remaining: 18h 17m 56s
8:	learn: 102.4187273	total: 3h 46m 23s	remaining: 17h 11m 21s
9:	learn: 109.7105152	total: 4h 13m 3s	remaining: 16h 52m 14s
10:	learn: 108.2195158	total: 4h 39m 23s	remaining: 16h 30m 34s
11:	learn: 109.7171448	total: 5h 5m 39s	remaining: 16h 7m 55s
12:	learn: 111.6882074	total: 5h 32m 2s	remaining: 15h 45m 1s
13:	learn: 118.7778246	total: 5h 58m 20s	remaining: 15h 21m 25s
14:	learn: 118.2325848	total: 6h 24m 53s	remaining: 14h 58m 4s
15:	learn: 115.5255909	total: 6h 51m 16s	remaining: 14h 

In [15]:
y_train_predicted = boosting_model.predict(X_train)
y_test_predicted = boosting_model.predict(X_test)

In [16]:
from sklearn.metrics import recall_score

In [34]:
recall_score(y_test, y_test_predicted, average='micro')

0.030370523131917234

In [18]:
pip install forge prophet

Note: you may need to restart the kernel to use updated packages.


In [19]:
from prophet import Prophet

In [20]:
df_t = data[data.columns[9:11]]
df_t.head()

,pass_dttm,time_to_under
0,2022-09-12 05:00:13,216.316667
1,2022-09-12 05:00:54,648.183333
2,2022-09-12 05:00:55,865.333333
3,2022-09-12 05:01:13,1048.233333
4,2022-09-12 05:02:55,965.600000


In [21]:
df_t.set_index('pass_dttm', inplace=True)
df_t.index = pd.to_datetime(df_t.index)
df_t.head()

,time_to_under
pass_dttm,
2022-09-12 05:00:13,216.316667
2022-09-12 05:00:54,648.183333
2022-09-12 05:00:55,865.333333
2022-09-12 05:01:13,1048.233333
2022-09-12 05:02:55,965.600000


In [22]:
split_date = '2022-09-18'
df_t_train = df_t.loc[df_t.index <= split_date].copy()
df_t_test = df_t.loc[df_t.index > split_date].copy()

In [23]:
df_t_train.reset_index() \
    .rename(columns={'pass_dttm':'ds',
                     'time_to_under':'y'}).head()

,ds,y
0,2022-09-12 05:00:13,216.316667
1,2022-09-12 05:00:54,648.183333
2,2022-09-12 05:00:55,865.333333
3,2022-09-12 05:01:13,1048.233333
4,2022-09-12 05:02:55,965.600000


In [24]:
model = Prophet()
model.fit(df_t_train.reset_index() \
    .rename(columns={'pass_dttm':'ds',
                     'time_to_under':'y'}))

22:10:00 - cmdstanpy - INFO - Chain [1] start processing
22:16:30 - cmdstanpy - INFO - Chain [1] done processing


In [25]:
df_t_test_model = model.predict(df=df_t_test.reset_index() \
                                   .rename(columns={'pass_dttm':'ds'}))

In [26]:
from sklearn.metrics import r2_score

In [27]:
r2_score(y_true=df_t_test['time_to_under'],y_pred=df_t_test_model['yhat'])

0.4782055732450359

In [28]:
result = 0.5*recall_score(y_test, y_test_predicted, average='micro')+0.5*r2_score(y_true=df_t_test['time_to_under'],y_pred=df_t_test_model['yhat'])
print(result)

0.25428804818847656
